<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Descente-de-Gradient" data-toc-modified-id="Descente-de-Gradient-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Descente de Gradient</a></span></li></ul></div>

Descente de Gradient
================

L'[algorithme de la descente de gradient](http://en.wikipedia.org/wiki/Gradient_descent) est un algorithme d'optimisation pour trouver un minimum local d'une fonction scalaire à partir d'un point donné, en effectuant de pas successifs dans la direction de l'inverse du gradient.

Pour une fonction $f: \mathbb{R}^n \to \mathbb{R}$, partant d'un point $\mathbf{x}_0$, la méthode calcule les points successifs dans le domaine de la fonction

$$
 \mathbf{x}_{n + 1} = \mathbf{x}_n - \eta \left( \nabla f \right)_{\mathbf{x}_n} \; ,
$$

où 

$\eta > 0$ est une taille de /pas/ suffisamment petite et  and $\left( \nabla f \right)_{\mathbf{x}_n}$ est le [gradient](http://en.wikipedia.org/wiki/Gradient) de $f$ évaluée au point $\mathbf{x}_n$. Les valeurs successives de la fonction 

$$
 f(\mathbf{x}_0) \ge f(\mathbf{x}_1) \ge f(\mathbf{x}_2) \ge \dots
$$

vont décroître globalement et la séquence $\mathbf{x}_n$ converge habituellement vers un minimum local.

En pratique utiliser un pas de taille fixe $\eta$ est particulièrement inefficace et la plupart des algorithmes vont plutôt chercher à l'adapter à chaque itération.

Le code suivant implémente la descente de gradient avec un pas de taille fixe s'arrétant quand la [norme](http://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm) du gradient descend en dessous d'un certain seuil.

Attention par défaut, pytorch *accumule* les gradients à chaque passe inverse!
C'est pourquoi il faut le remettre à zéro à chaque itération.

Commençons par importer les suspects usuels

In [1]:
import torch
import numpy as np
import math

Illustrons l'accumulation du gradient

In [2]:
x1 = torch.empty(2, requires_grad=True)
x1

tensor([0., 0.], requires_grad=True)

In [3]:
f1 = torch.pow(x1[0],2)
f1

tensor(0., grad_fn=<PowBackward0>)

In [4]:
# x1.grad.zero_()
f1.backward(retain_graph=True)
x1.grad

tensor([0., 0.])

In [7]:
x1.data.sub_(torch.ones(2))

tensor([-1., -1.])

tensor(0., grad_fn=<PowBackward0>)

Maintenant essayons d'implémenter une descente de gradient pour la fonction
$f(X) = sin(x_1) + cos(x_2)$ 

In [2]:
x0 = torch.ones(2,requires_grad=True)

In [3]:
f = torch.sin(x0[0]) + torch.cos(x0[1])
f

tensor(1.3818, grad_fn=<AddBackward0>)

On va avoir besoin de :
```python
f.backward(...) # Pour le calcul du gradient proprement dit
x.grad.data.zero_() # pour la remise à zéro du gradient après une itération
np.linalg.norm(x.grad.numpy()) # pour contrôler la convergence (norme l2)
```

On veut une fonction gd qui prend en argument $f, x, \eta, \epsilon$

In [5]:
def gd(f, x, eta, epsilon):
    while 1:
        f.backward(retain_graph=True)
#        print(np.linalg.norm(x.grad.numpy()))
        if (np.linalg.norm(x.grad.numpy()) < epsilon): 
            break
        else:
            x.data.sub_(eta * x.grad.data)
            x.grad.data.zero_()

In [6]:
gd(f, x0, 0.9, 0.00001)

In [7]:
print(x0.data)
print(f.data)

tensor([-1.5708,  3.1416])
tensor(1.3818)


Cette fonction ne permet pas d'avoir la valeur de $f$ directement sur le résultat. Il vaut mieux utiliser une fonction qu'un noeud de notre graphe  comme argument de notre descente de gradient.

In [8]:
x0 = torch.ones(2,requires_grad=True)
x0

tensor([1., 1.], requires_grad=True)

In [10]:
def f(x):
    return x[0].sin() + x[1].cos()

In [12]:
def gd(f, x, eta, epsilon):
    while 1:        
        f(x).backward()
#        print(np.linalg.norm(x.grad.numpy()))
        if (np.linalg.norm(x.grad.numpy()) < epsilon):
            break
        else:
            x.data.sub_(eta * x.grad.data)
            x.grad.data.zero_()

In [13]:
gd(f, x0, 0.9, 0.00001)

In [14]:
print(x0)
print(f(x0))

tensor([-1.5708,  3.1416], requires_grad=True)
tensor(-2., grad_fn=<AddBackward0>)
